In [31]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ok.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,Inola,OK,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",OKLAHOMA CITY,OK,...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,NaN
2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",TULSA,OK,...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,OKLAHOMA CITY,OK,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,Guymon,OK,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,Sapulpa,OK,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14
9993,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,Sallisaw,OK,...,999 Other,"2213 Water, Sewage and Other Systems",2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,NaN
9994,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,Oklahoma City,OK,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14
9995,722595,722595,9206568807,4/23/21,680,PPS,SAMURAI EQUIPMENT LLC,15437 E Pine St,Tulsa,OK,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,4.013105e+14


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [ ]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

In [ ]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [3]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/OK_fips_scraped.csv")


In [32]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/OK_fips_scraped.csv") 
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14


In [33]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [34]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [35]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [36]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1008918710,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1994657208,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,2427157100,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,6097937201,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,6262678403,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14
...,...,...,...,...,...
9992,8956648300,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14
9993,9099848507,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,4.013503e+14
9994,9183018301,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14
9995,9206568807,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,4.013105e+14


In [37]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() #5503

LoanNumber    22
full_add      22
Lat           22
Long          22
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [10]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [13]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('40')] #OK
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


  0%|          | 0/2536 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [14]:
geo_id_list =geo_id 

Length Check

In [15]:
print(len(geo_id_list))

2536


Matching FIPS to DataFrame

In [16]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2684] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [17]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_24349/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
1,1994657208,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,[401091066083007]
4,6262678403,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,[401399509021000]
11,7082757110,"9600 Broadway Extension, Oklahoma City, OK",35.568273,-97.510670,[401091083012002]
14,7195517006,"5801 N BROADWAY EXT STE 210, OKLAHOMA CITY, OK",35.536869,-97.514613,[401091062001030]
20,7343947107,"100 Enterprise B;vd, PAULS VALLEY, OK",34.744019,-97.245505,[400496813003024]
...,...,...,...,...,...
9966,2998527108,"400 S ACCESS RD, WEATHERFORD, OK",35.520860,-98.702742,[400399607021027]
9969,3676807307,"1611 S. I-35 Service Rd., Moore, OK",35.323557,-97.491330,[400272016043005]
9987,8006468301,"12000, MUSTANG, OK",35.384226,-97.724487,[400173014062029]
9993,9099848507,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,[401350301011054]


Remove brackets

In [18]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_24349/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
1,1994657208,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,401091066083007
4,6262678403,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,401399509021000
11,7082757110,"9600 Broadway Extension, Oklahoma City, OK",35.568273,-97.510670,401091083012002
14,7195517006,"5801 N BROADWAY EXT STE 210, OKLAHOMA CITY, OK",35.536869,-97.514613,401091062001030
20,7343947107,"100 Enterprise B;vd, PAULS VALLEY, OK",34.744019,-97.245505,400496813003024
...,...,...,...,...,...
9966,2998527108,"400 S ACCESS RD, WEATHERFORD, OK",35.520860,-98.702742,400399607021027
9969,3676807307,"1611 S. I-35 Service Rd., Moore, OK",35.323557,-97.491330,400272016043005
9987,8006468301,"12000, MUSTANG, OK",35.384226,-97.724487,400173014062029
9993,9099848507,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,401350301011054


In [19]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
64,4920347210,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,6223537005,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,3319208509,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,8230827006,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,4993557204,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,3053198401,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,9592567002,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2917617110,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2394,8825047105,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2550,5893907210,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


Parse for Merge

In [20]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1,1994657208,401091066083007
4,6262678403,401399509021000
11,7082757110,401091083012002
14,7195517006,401091062001030
20,7343947107,400496813003024
...,...,...
9966,2998527108,400399607021027
9969,3676807307,400272016043005
9987,8006468301,400173014062029
9993,9099848507,401350301011054


Read in base data

In [21]:
#if from geocode file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_ok.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/OK_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,Inola,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",OKLAHOMA CITY,...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,NaN
2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",TULSA,...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,OKLAHOMA CITY,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,Guymon,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,NaN


Merge with MainFrame

In [22]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,Inola,...,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14,NaN
1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",OKLAHOMA CITY,...,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,NaN,401091066083007
2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",TULSA,...,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14,NaN
3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,OKLAHOMA CITY,...,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14,NaN
4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,Guymon,...,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,NaN,401399509021000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9992,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,Sapulpa,...,8111 Automotive Repair and Maintenance,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14,NaN
9993,9993,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,Sallisaw,...,"2213 Water, Sewage and Other Systems",2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,NaN,401350301011054
9994,9994,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,Oklahoma City,...,3339 Other General Purpose Machinery Manufactu...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14,NaN
9995,9995,722595,722595,9206568807,4/23/21,680,PPS,SAMURAI EQUIPMENT LLC,15437 E Pine St,Tulsa,...,3329 Other Fabricated Metal Product Manufacturing,3329.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,4.013105e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [23]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,Inola,...,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14,NaN,401310505011056.0
1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",OKLAHOMA CITY,...,3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,NaN,401091066083007,401091066083007
2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",TULSA,...,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14,NaN,401430059001003.0
3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,OKLAHOMA CITY,...,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14,NaN,401091093003007.0
4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,Guymon,...,5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,NaN,401399509021000,401399509021000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9992,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,Sapulpa,...,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14,NaN,400370213002020.0
9993,9993,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,Sallisaw,...,2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,NaN,401350301011054,401350301011054
9994,9994,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,Oklahoma City,...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14,NaN,401091100003019.0
9995,9995,722595,722595,9206568807,4/23/21,680,PPS,SAMURAI EQUIPMENT LLC,15437 E Pine St,Tulsa,...,3329.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,4.013105e+14,NaN,401310504071023.0


Drop Extra FIPS Columns

In [24]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,Inola,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,401310505011056.0
1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",OKLAHOMA CITY,...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,401091066083007
2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",TULSA,...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,401430059001003.0
3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,OKLAHOMA CITY,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,401091093003007.0
4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,Guymon,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,401399509021000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9992,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,Sapulpa,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,400370213002020.0
9993,9993,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,Sallisaw,...,999 Other,"2213 Water, Sewage and Other Systems",2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,401350301011054
9994,9994,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,Oklahoma City,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,401091100003019.0
9995,9995,722595,722595,9206568807,4/23/21,680,PPS,SAMURAI EQUIPMENT LLC,15437 E Pine St,Tulsa,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,401310504071023.0


In [25]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
64,64,712664,712664,4920347210,4/27/20,680,PPP,DOWNSTREAM DEVELOPMENT AUTHORITY OF THE QUAPAW...,69300 Nee Rd,QUAPAW,...,999 Other,7211 Traveler Accommodation,7211.0,OKLAHOMA DISTRICT OFFICE,d $2-5 million,500 or more,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
423,423,713023,713023,6223537005,4/6/20,680,PPP,"J-M FARMS, INC",7001 S Hwy 69A,MIAMI,...,998 All Other Miscellaneous Manufacturing,1119 Other Crop Farming,1119.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,100 to 249,"7001 S Hwy 69A, MIAMI, OK",35.078999,-106.715022,NaN
491,491,713091,713091,3319208509,2/23/21,680,PPS,NORTHEASTERN TRIBAL HEALTH SYSTEM,7600 S Highway 69A,Miami,...,999 Other,6214 Outpatient Care Centers,6214.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,100 to 249,"7600 S Highway 69A, Miami, OK",37.196334,-94.191716,NaN
573,573,713173,713173,8230827006,4/8/20,680,PPP,NORTHEASTERN TRIBAL HEALTH SYSTEM,7600 S Hwy 69 A,MIAMI,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,100 to 249,"7600 S Hwy 69 A, MIAMI, OK",37.196334,-94.191716,NaN
644,644,713244,713244,4993557204,4/27/20,680,PPP,"SARACEN DEVELOPMENT, LLC",69300 Nee Rd,QUAPAW,...,999 Other,7211 Traveler Accommodation,7211.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,100 to 249,"69300 Nee Rd, QUAPAW, OK",37.000289,-94.621635,NaN
1009,1009,713609,713609,3053198401,2/4/21,680,PPS,SUMMIT LAYDOWN SERVICE INC,5705 US HIGHWAY 412,WOODWARD,...,999 Other,2131 Support Activities for Mining,2131.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,50 to 99,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
1010,1010,713610,713610,9592567002,4/9/20,680,PPP,"SUMMIT LAYDOWN SERVICE, INC",5705 US HIGHWAY 412,WOODWARD,...,999 Other,2131 Support Activities for Mining,2131.0,OKLAHOMA DISTRICT OFFICE,c $1-2 million,50 to 99,"5705 US HIGHWAY 412, WOODWARD, OK",36.071071,-90.237225,NaN
2210,2210,714810,714810,2917617110,4/11/20,680,PPP,ELMER SMITH OIL COMPANY,1715 South Highway 183,Clinton,...,999 Other,4247 Petroleum and Petroleum Products Merchant...,4247.0,OKLAHOMA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1715 South Highway 183, Clinton, OK",30.220229,-97.683076,NaN
2394,2394,714994,714994,8825047105,4/15/20,680,PPP,SEILNG MUNICIPAL HOSPTAL AUTHORITY,809 NE HWY 60 809 NE HWY 60,SEILING,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,OKLAHOMA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"809 NE HWY 60 809 NE HWY 60, SEILING, OK",41.285247,-99.385728,NaN
2550,2550,715150,715150,5893907210,4/27/20,680,PPP,HARPER COUNTY COMMUNITY HOSPITAL,1003 HWY 64 NORTH,BUFFALO,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,OKLAHOMA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1003 HWY 64 NORTH, BUFFALO, OK",35.922512,-75.678195,NaN


Convert to File

In [28]:
fips_OK = fips_merge_drop
#fips_TN = fips_merge_drop

In [29]:
fips_OK.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/OK_fips_scraped.csv") 

Review & Compare

In [30]:
fips_OK = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/OK_fips_scraped.csv") 
fips_OK

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,712600,712600,1008918710,3/26/21,680,PPP,PARFAB FIELD SERVICES LLC,15615 E 590 Rd,...,999 Other,2371 Utility System Construction,2371.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,100 to 249,"15615 E 590 Rd, Inola, OK",36.162500,-95.554756,4.013105e+14
1,1,1,712601,712601,1994657208,4/15/20,680,PPP,"LSB INDUSTRIES, INC","3503 NW 63rd Street, Suite 500",...,325 Chemicals,"3253 Pesticide, Fertilizer, and Other Agricult...",3253.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"3503 NW 63rd Street, Suite 500, OKLAHOMA CITY, OK",35.537785,-97.579151,4.010911e+14
2,2,2,712602,712602,2427157100,4/10/20,680,PPP,"LATSHAW DRILLING COMPANY, LLC","4500 S. 129th E Ave, Suite 150",...,999 Other,2111 Oil and Gas Extraction,2111.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,250 to 499,"4500 S. 129th E Ave, Suite 150, TULSA, OK",36.217976,-95.833200,4.014301e+14
3,3,3,712603,712603,6097937201,4/27/20,680,PPP,"HAC, INC",390 NW 36TH ST,...,999 Other,4451 Grocery Stores,4451.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"390 NW 36TH ST, OKLAHOMA CITY, OK",35.507790,-97.519860,4.010911e+14
4,4,4,712604,712604,6262678403,2/10/21,680,PPP,QUALITY INTEGRATED SERVICES INC.,2309 N Lelia St,...,999 Other,"5419 Other Professional, Scientific, and Techn...",5419.0,OKLAHOMA DISTRICT OFFICE,e $5-10 million,500 or more,"2309 N Lelia St, Guymon, OK",36.703597,-101.477175,4.013995e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9992,9992,722592,722592,8956648300,1/30/21,680,PPS,JS&J ENTERPRISES LLC,205 S Main St,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"205 S Main St, Sapulpa, OK",35.996254,-96.114040,4.003702e+14
9993,9993,9993,722593,722593,9099848507,3/12/21,680,PPP,THE SEQUOYAH COUNTY WATER ASSOCIATION INC.,3520 W. Cherokee,...,999 Other,"2213 Water, Sewage and Other Systems",2213.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3520 W. Cherokee, Sallisaw, OK",35.463139,-94.837986,4.013503e+14
9994,9994,9994,722594,722594,9183018301,1/30/21,680,PPS,WILKIE MFG LLC,2640 NW 2nd St,...,333 Machinery,3339 Other General Purpose Machinery Manufactu...,3339.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2640 NW 2nd St, Oklahoma City, OK",35.470173,-97.559910,4.010911e+14
9995,9995,9995,722595,722595,9206568807,4/23/21,680,PPS,SAMURAI EQUIPMENT LLC,15437 E Pine St,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,OKLAHOMA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"15437 E Pine St, Tulsa, OK",36.177074,-95.804920,4.013105e+14
